In [282]:
import pandas as pd
import numpy as np
import io
from io import StringIO

Imagina que eres un investigador en el mundo de Pokémon y has recibido una base de datos con información desactualizada y desordenada sobre avistamientos de Pokémon. Tu misión es limpiar y actualizar esta base de datos para que pueda ser utilizada en un estudio sobre la población de Pokémon en la región.

Datos Iniciales

Los datos iniciales contienen las siguientes columnas:

    SightingDate: Fecha del avistamiento.
    TrainerID: Identificación del entrenador que reportó el avistamiento.
    PokemonName: Nombre del Pokémon avistado.
    CP: Puntos de combate del Pokémon reportado.
    HP: Puntos de salud del Pokémon reportado.
    Type: Tipo del Pokémon.
    Weather: Clima durante el avistamiento.

# 1 Carga de Datos

In [283]:
# Obtenemos el encoding de usando chardet
import chardet

path_datos = 'pokemon.csv'

with open(path_datos, "rb") as f:
    result = chardet.detect(f.read())
    
encoding = result["encoding"]
print("Encoding detectado: ", encoding)

#Creamos el DataFrame con el encoding detectado
df = pd.read_csv(path_datos, encoding = encoding)

df

Encoding detectado:  ISO-8859-1


,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather
0,2023-11-08T14:00:00+0000,2023-11-08T14:00Z,TR123,Pikachu,500,35,Electric,Clear
1,2023-07-12T09:30:00+0100,2023-07-12T08:30Z,TR456,Eevee,MISSING,55,Normal,Cloudy
2,2023/02/23T22:15:00+0900,2023-02-23T13:15Z,TR789,Magicarp,"1,000",10,Water,Rain
3,2023-04-30T06:45:00-0400,2023-04-30T10:45Z,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY
4,2023-08-15T16:00:00+1000,NaN,TR102,Bulbasaur,750,50,Grass/Poison,Sunny


# Limpieza de Datos

## 1 Normalizacion de Zonas Horarias

Normaliza la columna `'SightingTimeUTC'` a la zona horaria UTC y convierte `'SightingDate'` al mismo formato de tiempo.


In [284]:
# Vemos cuales son los formatos de las fechas antes de normalizar
print(f"Formatos de fecha originales: SightingTimeUTC: {df['SightingTimeUTC'].dtype}, SightingDate: {df['SightingDate'].dtype}")

# Hacemos la normalización
df['SightingTimeUTC'] = pd.to_datetime(df['SightingDate'], errors = 'coerce', utc = True, format = 'mixed')
df['SightingDate'] = pd.to_datetime(df['SightingDate'], errors = 'coerce', utc = True, format = 'mixed')

# Corroboramos que se haya cambiado el tipo de fecha correctamente
print(f"Formatos de fecha finales: SightingTimeUTC: {df['SightingTimeUTC'].dtype}, SightingDate: {df['SightingDate'].dtype}")

df

Formatos de fecha originales: SightingTimeUTC: object, SightingDate: object
Formatos de fecha finales: SightingTimeUTC: datetime64[ns, UTC], SightingDate: datetime64[ns, UTC]


,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500,35,Electric,Clear
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,MISSING,55,Normal,Cloudy
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,"1,000",10,Water,Rain
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750,50,Grass/Poison,Sunny


Compara si la fecha de la columna `'SightingDate'` coincide con la fecha en `'SightingTimeUTC'` una vez normalizada.


In [285]:
# Creamos una nueva columna booleana que resulta de la comparación de la fecha de 'SightingDate' con 'SightingTimeUTC' 
df['DateMatch'] = df['SightingDate'].dt.date == df['SightingTimeUTC'].dt.date
df


,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500,35,Electric,Clear,True
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,MISSING,55,Normal,Cloudy,True
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,"1,000",10,Water,Rain,True
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY,True
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750,50,Grass/Poison,Sunny,True


Ajusta `'SightingTimeUTC'` a la zona horaria local de cada entrenador y crea una columna `'SightingTimeLocal'`.

In [286]:
# Dado que SightingTimeUTC contiene la zona horaria, la hora local puede obtenerse de la siguiente forma:
df['SightingTimeLocal'] = df['SightingTimeUTC'].dt.tz_localize(None)
df


,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500,35,Electric,Clear,True,2023-11-08 14:00:00
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,MISSING,55,Normal,Cloudy,True,2023-07-12 08:30:00
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,"1,000",10,Water,Rain,True,2023-02-23 13:15:00
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750,50,Grass/Poison,Sunny,True,2023-08-15 06:00:00


Calcula el tiempo transcurrido desde el momento del avistamiento hasta `'ahora'` (tu hora local) y crea una columna `'TimeSinceSighting'`.

In [287]:
# Para poder obtener la diferencia, necesito el método now() de datetime
from datetime import datetime 
df['TimeSinceSighting'] = datetime.now() - df['SightingTimeLocal']
df

,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal,TimeSinceSighting
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500,35,Electric,Clear,True,2023-11-08 14:00:00,2 days 01:44:06.938229
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,MISSING,55,Normal,Cloudy,True,2023-07-12 08:30:00,121 days 07:14:06.938229
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,"1,000",10,Water,Rain,True,2023-02-23 13:15:00,260 days 02:29:06.938229
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00,194 days 04:59:06.938229
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750,50,Grass/Poison,Sunny,True,2023-08-15 06:00:00,87 days 09:44:06.938229


## 2. Limpeiza de IDs

Llena los valores faltantes en `'TrainerID'` con el ID `'UNKNOWN'`.


In [288]:
# Para poder hacer esto, podemos usar el método fillna
df['TrainerID'] = df['TrainerID'].fillna('UNKNOWN')
df


,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal,TimeSinceSighting
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500,35,Electric,Clear,True,2023-11-08 14:00:00,2 days 01:44:06.938229
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,MISSING,55,Normal,Cloudy,True,2023-07-12 08:30:00,121 days 07:14:06.938229
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,"1,000",10,Water,Rain,True,2023-02-23 13:15:00,260 days 02:29:06.938229
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00,194 days 04:59:06.938229
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750,50,Grass/Poison,Sunny,True,2023-08-15 06:00:00,87 days 09:44:06.938229


## 3. Corrección de Nombres de Pokémon

Asegúrate de que los nombres de Pokémon estén capitalizados correctamente.


In [289]:
# Tomaré como capitalizado correctamente que tenga la primera letra mayuscula y las demas minusculas

# Usamos el metodo capitalize de la clase String
df['PokémonName'] = df['PokémonName'].str.capitalize()
df

,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal,TimeSinceSighting
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500,35,Electric,Clear,True,2023-11-08 14:00:00,2 days 01:44:06.938229
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,MISSING,55,Normal,Cloudy,True,2023-07-12 08:30:00,121 days 07:14:06.938229
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,"1,000",10,Water,Rain,True,2023-02-23 13:15:00,260 days 02:29:06.938229
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00,194 days 04:59:06.938229
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750,50,Grass/Poison,Sunny,True,2023-08-15 06:00:00,87 days 09:44:06.938229


## 4. Conversión de 'CP' y 'HP' a Numéricos

Convierte `'CP'` y `'HP'` a valores numéricos, manejando los `'MISSING'` y comas como separadores de miles.

In [290]:
# Observamos los tipos de datos de ambas columnas
print(f"Tipos de datos originales de las columnas: \nCP: {df['CP'].dtype}, HP: {df['HP'].dtype}")

Tipos de datos originales de las columnas: 
CP: object, HP: int64


In [291]:

# Quiero convertir los datos a int64. pero tengo un MISSING y comas que hacen que no lo pueda manejar

# HP ya es int64, por lo que no modificaré nada.
# Sin embargo, en CP tengo un MISSING y comas que hacen que no lo pueda manejar
 
# Primero convertimos a String
df['CP'] = df['CP'].astype(str)

# Quito las comas
df['CP'] = df['CP'].str.replace(',', '')

# Uso el método to_numeric para cambiar a numérico. Como hay un Nan, será float64
df['CP'] = pd.to_numeric(df['CP'], errors = 'coerce')

#Corroboro los tipos de datos finales
print(f"Tipos de datos finales de las columnas: \nCP: {df['CP'].dtype}, HP: {df['HP'].dtype}")
df

Tipos de datos finales de las columnas: 
CP: float64, HP: int64


,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal,TimeSinceSighting
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500.0,35,Electric,Clear,True,2023-11-08 14:00:00,2 days 01:44:06.938229
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,NaN,55,Normal,Cloudy,True,2023-07-12 08:30:00,121 days 07:14:06.938229
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,1000.0,10,Water,Rain,True,2023-02-23 13:15:00,260 days 02:29:06.938229
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800.0,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00,194 days 04:59:06.938229
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750.0,50,Grass/Poison,Sunny,True,2023-08-15 06:00:00,87 days 09:44:06.938229


## 5. Estandarización de 'Type'

Divide la columna `'Type'` en `'PrimaryType'` y `'SecondaryType'` cuando hay dos tipos.


In [292]:
# Separamos en dos columnas usando el método split.
df[['PrimaryType', 'SecondaryType']] = df['Type'].str.split('/', expand = True)
df

,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal,TimeSinceSighting,PrimaryType,SecondaryType
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500.0,35,Electric,Clear,True,2023-11-08 14:00:00,2 days 01:44:06.938229,Electric,None
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,NaN,55,Normal,Cloudy,True,2023-07-12 08:30:00,121 days 07:14:06.938229,Normal,None
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,1000.0,10,Water,Rain,True,2023-02-23 13:15:00,260 days 02:29:06.938229,Water,None
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800.0,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00,194 days 04:59:06.938229,Ghost,None
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750.0,50,Grass/Poison,Sunny,True,2023-08-15 06:00:00,87 days 09:44:06.938229,Grass,Poison


## 6. Corrección del Clima

Estándariza la columna `'Weather'` para que todos los valores sean mayúsculas.


In [293]:
# Usaremos el método upper de la clase String
df['Weather'] = df['Weather'].str.upper()
df

,SightingDate,SightingTimeUTC,TrainerID,PokémonName,CP,HP,Type,Weather,DateMatch,SightingTimeLocal,TimeSinceSighting,PrimaryType,SecondaryType
0,2023-11-08 14:00:00+00:00,2023-11-08 14:00:00+00:00,TR123,Pikachu,500.0,35,Electric,CLEAR,True,2023-11-08 14:00:00,2 days 01:44:06.938229,Electric,None
1,2023-07-12 08:30:00+00:00,2023-07-12 08:30:00+00:00,TR456,Eevee,NaN,55,Normal,CLOUDY,True,2023-07-12 08:30:00,121 days 07:14:06.938229,Normal,None
2,2023-02-23 13:15:00+00:00,2023-02-23 13:15:00+00:00,TR789,Magicarp,1000.0,10,Water,RAIN,True,2023-02-23 13:15:00,260 days 02:29:06.938229,Water,None
3,2023-04-30 10:45:00+00:00,2023-04-30 10:45:00+00:00,TR101,Gengar,800.0,45,Ghost,PARTLY_CLOUDY,True,2023-04-30 10:45:00,194 days 04:59:06.938229,Ghost,None
4,2023-08-15 06:00:00+00:00,2023-08-15 06:00:00+00:00,TR102,Bulbasaur,750.0,50,Grass/Poison,SUNNY,True,2023-08-15 06:00:00,87 days 09:44:06.938229,Grass,Poison


# 3 Analisis de Datos

## Agrupaciones

Agrupa el DataFrame por `'Type'` y calcula la suma de `'CP'` para cada grupo.

In [294]:
grouped = df.groupby('Type')['CP'].sum().reset_index()
grouped

,Type,CP
0,Electric,500.0
1,Ghost,800.0
2,Grass/Poison,750.0
3,Normal,0.0
4,Water,1000.0


Después de la suma, agrega una columna que calcule la media de `'HP'` por cada `'Type'`, pero solo para aquellos Pokémon cuyo `'CP'` sea mayor que el promedio de `'CP'` de todo el DataFrame.

In [295]:
# Primero obtenemos el promedio de CP
promedio_cp = df['CP'].mean()

# Filtramos el DataFrame original
df_filtrado = df[df['CP'] > promedio_cp]

# Al dataframe anterior agregamos la media por type
filtro = df_filtrado.groupby('Type')['HP'].mean()

final = pd.merge(grouped, filtro, on = 'Type', how = 'left').reset_index()
final

,index,Type,CP,HP
0,0,Electric,500.0,NaN
1,1,Ghost,800.0,45.0
2,2,Grass/Poison,750.0,NaN
3,3,Normal,0.0,NaN
4,4,Water,1000.0,10.0
